In [22]:
# Import necessary modules
import os
import requests
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv  # To load environment variables from the .env file

In [23]:
# Load environment variables from .env
load_dotenv()

True

In [24]:
# Read the API key from the environment variables
ALPHAVANTAGE_KEY = os.getenv('ALPHAVANTAGE_KEY')
if ALPHAVANTAGE_KEY is None:
    raise Exception("ALPHAVANTAGE_KEY not set in the .env file.")

In [33]:
def get_daily_prices(symbol):
    """
    Fetches daily time series data for the provided symbol from AlphaVantage API.
    
    Parameters:
        symbol (str): The stock symbol to fetch data for (e.g., 'AAPL').
        
    Returns:
        pd.DataFrame: DataFrame containing the time series data with dates and prices.
    """
    url = "https://www.alphavantage.co/query"
    params = {
        'function': 'TIME_SERIES_DAILY',  # Changed from TIME_SERIES_DAILY_ADJUSTED
        'symbol': symbol,
        'outputsize': 'compact',  # Returns the latest 100 data points
        'apikey': ALPHAVANTAGE_KEY
    }
    response = requests.get(url, params=params)
    response.raise_for_status()
    data = response.json()
    
    # Print the actual response to help debug
    print("API Response Keys:", data.keys())
    
    # Check if there's an error message
    if "Error Message" in data:
        raise Exception(f"API Error: {data['Error Message']}")
    
    # Check if we're being rate limited
    if "Note" in data:
        raise Exception(f"API Note: {data['Note']}")
    
    # The key for daily data is "Time Series (Daily)"
    time_series = data.get("Time Series (Daily)")
    if not time_series:
        raise Exception(f"No time series data returned from API for symbol: {symbol}")
    
    return time_series

In [34]:
# Try the API call again
symbol = "AAPL"
try:
    time_series = get_daily_prices(symbol)
    print("\nSuccess! First few dates:")
    print(list(time_series.items())[:3])
except Exception as e:
    print(f"Error: {e}")

API Response Keys: dict_keys(['Meta Data', 'Time Series (Daily)'])

Success! First few dates:
[('2025-04-07', {'1. open': '177.2000', '2. high': '194.1500', '3. low': '174.6200', '4. close': '181.4600', '5. volume': '160466286'}), ('2025-04-04', {'1. open': '193.8900', '2. high': '199.8800', '3. low': '187.3400', '4. close': '188.3800', '5. volume': '125910913'}), ('2025-04-03', {'1. open': '205.5400', '2. high': '207.4900', '3. low': '201.2500', '4. close': '203.1900', '5. volume': '103419006'})]
